In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ibllib.io.spikeglx import Reader
import estimate_displacement as ed
from utils import mat2npy

In [2]:
# This implementation has been tested with Neuropixels 1.0
geomarray = mat2npy('neuropixPhase3B1_kilosortChanMap.mat') # convert .mat chan file to .npy chan file

In [3]:
# list of base directories containing neuropixel data folders
base_dirs = ['']
# filename list
file_list = ['']
# horizontal (time) smoothing list. If you have jumps in the registration you can increase this value to smooth estimation over time (>50)
horz_smooth = [7]

done_ind = [] # indices we've already processed
for index in sorted(done_ind, reverse=True):
    del base_dirs[index]
    del file_list[index]
    del horz_smooth[index]

print(file_list)

# I've only tested the spikeglx data reader that's part of ibllib (pip install ibllib)
# yass is the default reader, but I've removed any mandatory yass imports in case you don't have that
reader_type = 'spikeglx'
# We only have to detect spikes once per dataset, then we can run the registration multiple times to test parameters
detect_spikes = True
# I've found non-rigid registration to be optimal, but it can introduce artifacts for some datasets
rigid = False
if rigid:
    reg_win_num = 1
    reg_block_num = 1
    registration_type = 'rigid'
else:
    reg_win_num = 5 # default windowing
    reg_block_num = 10 # default registration blocks
    registration_type = 'non_rigid'

for i in range(len(file_list)):
    file_name = file_list[i]
    working_directory = base_dirs[i] + '/Neuropixel-' + file_name + '/' + file_name + '_g0/' + file_name + '_g0_imec0/'
    registration_directory = working_directory + 'NeuropixelsRegistration/'
    if not os.path.exists(registration_directory):
        os.makedirs(registration_directory)

    # Prepare the data loader
    reader = Reader(working_directory + file_name + '_g0_tcat.imec0.ap.bin') # I assume you've used CatGT (tcat / t0)

    if detect_spikes:
        # detect spikes
        raster = ed.check_raster(reader, geomarray, reader_type=reader_type, num_chans_per_spike=2,
                             detection_threshold=6,
                             working_directory=registration_directory,
                             save_raster_info=True)
        # view raster
        plt.figure(figsize=(16, 10))
        plt.imshow(raster, vmin=0, vmax=10, aspect="auto", cmap=plt.get_cmap('inferno'))
        plt.ylabel("depth", fontsize=16)
        plt.xlabel("time", fontsize=16)
        plt.colorbar()
        plt.show()

    # run the registration
    # visualization will be saved to decentralized_raster/, shift will be saved as total_shift.npy
    total_shift = ed.estimate_displacement(reader, geomarray,
                                           reader_type=reader_type,
                                           num_chans_per_spike=2,
                                           detection_threshold=6,
                                           horz_smooth=horz_smooth[i],
                                           reg_win_num=reg_win_num,
                                           reg_block_num=reg_block_num,
                                           iteration_num=4,
                                           resume_with_raster=True,
                                           working_directory=registration_directory)

    plt.imshow(total_shift, aspect='auto')
    plt.colorbar()
    plt.show()

    # create a new binary file with the drift corrected data ('standardized.bin')
    # this file does not contain the digital sync channel, so use your original file for that
    ed.register(reader, geomarray, total_shift, reader_type=reader_type,
                registration_type=registration_type,
                working_directory=registration_directory)

['']


PermissionError: [Errno 13] Permission denied: '/Neuropixel-'

In [ ]:
# Here I've broken up a few of the steps into cells
base_dir = ''
file_list = ['']
reader_type = 'spikeglx'
for i in range(len(file_list)):
    file_name = file_list[i]
    working_directory = base_dir + 'Neuropixel-' + file_name + '/' + file_name + '_g0/' + file_name + '_g0_imec0/'
    registration_directory = working_directory + 'NeuropixelsRegistration/'
    reader = Reader(working_directory + file_name + '_g0_tcat.imec0.ap.bin')

In [ ]:
total_shift = ed.estimate_displacement(reader, geomarray,
                                           reader_type=reader_type,
                                           num_chans_per_spike=2,
                                           detection_threshold=6,
                                           horz_smooth=7,
                                           reg_win_num=5,
                                           reg_block_num=10,
                                           iteration_num=4,
                                           resume_with_raster=True,
                                           working_directory=registration_directory)

In [ ]:
total_shift = np.load(registration_directory + 'total_shift.npy')
plt.imshow(total_shift, aspect='auto')
plt.colorbar()
plt.show()

In [ ]:
ed.register(reader, geomarray, total_shift, reader_type=reader_type,
                registration_type='non_rigid',
                working_directory=registration_directory)